In [2]:
import pandas as pd
import numpy as np

In [ ]:
# generate two data frames containing all of the listing and calendar data for each location
import os
import pandas as pd

directory_path = 'home_rentals/'

# Initialize an empty list to store DataFrames
listing_df = []
calendar_df = []

# Iterate through subdirectories
for subdirectory in os.listdir(directory_path):
    subdirectory_path = os.path.join(directory_path, subdirectory) 

    # Check if the subdirectory contains a "listings.csv.gz" file
    csv_listing_file_path = os.path.join(subdirectory_path, "listings.csv.gz")
    csv_calendar_file_path = os.path.join(subdirectory_path, "calendar.csv.gz")
    print(csv_listing_file_path)
    print(csv_calendar_file_path)

    df1 = pd.read_csv(csv_listing_file_path)
    df2 = pd.read_csv(csv_calendar_file_path)

    df1['subdirectory_path'] = subdirectory_path
    df2['subdirectory_path'] = subdirectory_path

    listing_df.append(df1)
    calendar_df.append(df2)


# Concatenate all DataFrames into a single DataFrame
all_listing_df = pd.concat(listing_df, ignore_index=True)
all_calendar_df = pd.concat(calendar_df, ignore_index=True)

In [1]:
###### DELETE THIS IF THE BLOCK ABOVE WORKS!!! #######



# # alternative code to generate a big data frame with listing data for all locations 
# import os
# import pandas as pd

# directory_path = 'home_rentals/'

# # Initialize an empty list to store DataFrames
# dataframes = []

# # Iterate through subdirectories
# for subdirectory in os.listdir(directory_path):
#     subdirectory_path = os.path.join(directory_path, subdirectory) 

#     # Check if the subdirectory contains a "listings.csv.gz" file
#     csv_file_path = os.path.join(subdirectory_path, "listings.csv.gz")
#     print(csv_file_path)

#     df = pd.read_csv(csv_file_path)
#     dataframes.append(df)


# # Concatenate all DataFrames into a single DataFrame
# mega_dataframe = pd.concat(dataframes, ignore_index=True)

In [ ]:
# Make a new dataframe with the average availability per location, and generate a new data frame with the results

import pandas as pd

all_calendar_df["date"] = pd.to_datetime(all_calendar_df["date"])
all_calendar_df["month"] = all_calendar_df["date"].dt.month
all_calendar_df["year"] = all_calendar_df["date"].dt.year
all_calendar_df['month_year'] = all_calendar_df['month'].astype(str) + '-' + all_calendar_df['year'].astype(str)

# make a new data frame
available_by_month = (all_calendar_df[all_calendar_df['available'] == 't'].groupby(['listing_id', 'month_year']).size().unstack(fill_value=0).stack().reset_index(name='days_available'))

# Calculate the average days_available for each month_year (making sure that it rounds to full numbers)
average_by_month = (available_by_month.groupby('month_year')['days_available'].mean().round(0).reset_index(name='average_days_available'))

# convert the month_year column to datetime format 
average_by_month['month_year'] = pd.to_datetime(average_by_month['month_year'], format='%m-%Y').dt.to_period('M')

# sort the rows by ascending dates
average_by_month = average_by_month.sort_values('month_year')

# reset the row index
average_by_month.reset_index(drop=True, inplace=True)

average_by_month


In [ ]:
# check that each location has data for the same months

all_locations = ["Amsterdam", "Barcelona", "Euskadi", "Geneva", "Ireland", "Lisbon", "Los Angeles", "Madrid", "Malaga", "Mallorca", "Malta", "Menorca", "Mexico City", "Milan", "Naples", 
                 "Rome", "Rotterdam", "San Diego", "San Francisco", "Tokyo", "Toronto", "Valencia", "Vancouver", "Vienna", "Washington", "Winnipeg", "Zurich"]


for location in all_locations:
    one_loc = average_by_month[average_by_month["location"] == location]
    all_dates = one_loc["month_year"].unique()
    print(location, all_dates)




Amsterdam <PeriodArray>
['2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
 '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09']
Length: 13, dtype: period[M]
Barcelona <PeriodArray>
['2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
 '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09']
Length: 13, dtype: period[M]
Euskadi <PeriodArray>
['2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
 '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10']
Length: 14, dtype: period[M]
Geneva <PeriodArray>
['2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
 '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09']
Length: 13, dtype: period[M]
Ireland <PeriodArray>
['2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03',
 '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09']
Length: 13, dtype: period[M]
Lisbon

As can be seen above, all of the locations have data ranging from 2022-09 to 2023-09, except for Euskadi which has an extra month of data (2023-10). This is an additional month in comparison to the other locations, as such we will remove it to have a consistent range of data between all of the locations. In addition, we can assume that the data for the 2023-10 is incorrect since it predicts there the month has already been fully booked, but when looking at the spread of data for the previous months (almost all of them have at least half of the month unbooked) this seems highly unlikely. 

In [ ]:
# remove the additional month of data for Euskadi

# Create a boolean mask based on the specified conditions
mask = average_by_month['month_year'] == "2023-10"

# Use the boolean mask to filter the DataFrame and keep only the rows that don't match the conditions
average_by_month = average_by_month[~mask]

# check that we got the result we wanted
average_by_month[average_by_month["location"]=="Euskadi"]

,month_year,average_days_available,location
26,2022-09,2.0,Euskadi
27,2022-10,17.0,Euskadi
28,2022-11,20.0,Euskadi
29,2022-12,21.0,Euskadi
30,2023-01,19.0,Euskadi
31,2023-02,18.0,Euskadi
32,2023-03,19.0,Euskadi
33,2023-04,15.0,Euskadi
34,2023-05,16.0,Euskadi
35,2023-06,16.0,Euskadi


We  have now successfully generated a data frame with all of the data required to build our model.

In [5]:
''' 

TO DO FOR DATA CHECKING + VERIFICATION:
- check that there is no data where the number of available days > number of days in the month, and also maybe check if there are any
    locations with very strange distributions of availabilities over the months??
- check number of listings per location (shouldn't have any that have too few)
- go over the property_types and only pick the ones we're interested in (houses, apartments, etc.) and discard everything else  
    (e.g. windmills, etc.)  --> can also look at how frequently each type appears and only pick the top few types

TO DO FOR MODEL GENERATION:
- split the data set into test and train  
- etc. 
'''


" \n\nTO DO FOR DATA CHECKING + VERIFICATION:\n- check that there is no data where the number of available days > number of days in the month, and also maybe check if there are any\n    locations with very strange distributions of availabilities over the months??\n- check number of listings per location (shouldn't have any that have too few)\n- go over the property_types and only pick the ones we're interested in (houses, apartments, etc.) and discard everything else  \n    (e.g. windmills, etc.)  --> can also look at how frequently each type appears and only pick the top few types\n\nTO DO FOR MODEL GENERATION:\n- split the data set into test and train  \n- etc. \n"

In [ ]:
import pandas as pd 

amsterdamdata = pd.read_csv('home_rentals\Amsterdam\listings.csv.gz')
amsterdamdata["property_type"].unique()

proptypes = (amsterdamdata.groupby('property_type').count())

proptypes


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
property_type,,,,,,,,,,,,,,,,,,,,,
Barn,4,4,4,4,4,4,4,3,4,4,...,3,3,3,2,4,4,4,4,4,3
Boat,79,79,79,79,79,79,79,52,79,79,...,74,74,74,78,79,79,79,79,79,74
Camper/RV,4,4,4,4,4,4,4,0,4,4,...,2,2,2,3,4,4,4,4,4,2
Cave,1,1,1,1,1,1,1,0,1,1,...,0,0,0,1,1,1,1,1,1,0
Entire bungalow,5,5,5,5,5,5,5,3,5,5,...,4,4,4,2,5,5,5,5,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tent,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1
Tiny home,18,18,18,18,18,18,17,14,18,18,...,16,16,16,6,18,18,18,18,18,16
Tower,3,3,3,3,3,3,3,2,3,3,...,3,3,3,3,3,3,3,3,3,3


## Data Anaysis


In [ ]:

city = available_all_loc['location'].unique()

NameError: name 'available_all_loc' is not defined

In [ ]:
file = pd.read_csv("home_rentals/Ireland/reviews.csv.gz")

In [ ]:
file

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,704148680943643330,715761219809350547,2022-09-14,415093097,Guillaume,Louise and her buddy Bohdi are wonderful hosts...
1,606997848080805457,628030426065591294,2022-05-16,427350855,Taynan,Kate Apartament is sparkling clean . She is ea...
2,606997848080805457,631033961631011367,2022-05-20,263380539,Niamh,"Really nice apartment, with everything you nee..."
3,606997848080805457,635363398272267326,2022-05-26,96164395,Jackie,"Great location, spacious apartment and good su..."
4,606997848080805457,637489065335555111,2022-05-29,62628139,Peter,"Good location, easy walking distance to the ma..."
...,...,...,...,...,...,...
1110056,37771431,519750208,2019-08-29,58672654,George,"It was a pleasant stay, due to Robert’s great ..."
1110057,37771431,525773396,2019-09-08,45141196,Esmond,Lovely staying with Robert and family. The roo...
1110058,37771431,530082145,2019-09-15,216358574,Aida,We had a really nice stay with Robert. The roo...
1110059,606130299805738050,691147324135233771,2022-08-11,403300357,Olivier,Appartement superbe et très bien localisé pour...
